## Import from GCS with Pandas 

In [ ]:
import tensorflow as tf
print(tf.version.VERSION)

In [ ]:
!pip install google-cloud-storage

In [ ]:
!pip install gcsfs

In [ ]:
import os
from google.cloud import storage
import pandas as pd
  
# create an Empty DataFrame object
df = pd.DataFrame()
temp_df = pd.DataFrame()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/notebooks/flawless-shard-380118-6483865e92cb.json"
bucket_name = "bci_bucket"

# def list_blobs_with_prefix(bucket_name, prefix,delimiter=None):
storage_client = storage.Client() 
blobs = storage_client.list_blobs(bucket_name,prefix='data/',delimiter=None)

# init empty pdf
# df = pd.read_csv('gcs://your-bucket/path/data.csv.gz',
#                  storage_options={"token": "credentials.json"})
#add col label

# updf = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/up", inferSchema='false')

for b in blobs:
    label_value = os.path.split(b.name)[0]
    label_value = os.path.split(label_value)[1]
    
    temp_df = pd.read_csv(f"gs://bci_bucket/data/{label_value}/*.csv", sep='\t',
                     storage_options={"token": '/home/jovyan/notebooks/flawless-shard-380118-6483865e92cb.json'})
    df = pd.concat([df, temp_df])
    
    #into dataframe append b from gcs // tab delim
    #add label_value to col label
        
        # list_label.append(label_value)

In [ ]:
print(df.head(3))
print(df.shape)

In [ ]:
import pandas as pd
col = []
for x in range (0, 32):
    x = f'_c{x}'
    col.append(x)
df = pd.DataFrame(columns=col)

In [ ]:
testdf = pd.read_csv("/home/jovyan/notebooks/BrainFlow-RAW_Recordings_takeoff_4.csv", sep='\t', header=0, index_col=None)

In [ ]:
testdf.head(2)

In [ ]:
pd.concat([df, testdf])

## Import From GCS with Spark

In [ ]:
!pip install delta-spark==2.3.0

In [ ]:
import pyspark
from delta import *
from pyspark.sql.functions import lit
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("google.cloud.auth.service.account.json.keyfile", "/home/jovyan/notebooks/flawless-shard-380118-6483865e92cb.json")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

#### Writing to Delta lake

In [ ]:
updf = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/up", inferSchema='true')
takeoff = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/takeoff", inferSchema='true')
landdf = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/land", inferSchema='true')
downdf = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/down", inferSchema='true')
leftdf = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/left", inferSchema='true')
rightdf = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/right", inferSchema='true')
forward = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/forward", inferSchema='true')
backward = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/backward", inferSchema='true')

In [ ]:
updf = updf.withColumn("label", lit("up"))
takeoff = takeoff.withColumn("label", lit("takeoff"))
landdf = landdf.withColumn("label", lit("land"))
downdf = downdf.withColumn("label", lit("down"))
leftdf = leftdf.withColumn("label", lit("left"))
rightdf = rightdf.withColumn("label", lit("right"))
forward = forward.withColumn("label", lit("forward"))
backward = backward.withColumn("label", lit("backward"))

In [ ]:
df = updf.union(takeoff)
df = df.union(landdf)
df = df.union(downdf)
df = df.union(leftdf)
df = df.union(rightdf)
df = df.union(forward)
df = df.union(backward)

In [ ]:
df.write \
  .format("delta") \
  .mode("overwrite") \
  .save("gs://bci_bucket/delta-table-data/")

In [ ]:
df1 = spark.read.format("delta").load("gs://bci_bucket/delta-table-data/")

### Split into Test Train and Convert to Pandas

In [ ]:
pdf = df1.toPandas()

In [ ]:
pdf.head(9)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(pdf, test_size=0.1)

In [ ]:
train.head(3)

### Train Model with Spark to Pandas

In [ ]:
!pip install tensorflow==2.11.0

In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_decision_forests as tfdf
import pandas as pd

# Load a dataset in a Pandas dataframe.

# Convert the dataset into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="label")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="label")

In [ ]:
test.shape

In [ ]:
# Train a Random Forest model.
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

In [ ]:
# Summary of the model structure.
model.summary()

# Evaluate the model.
model.evaluate(test_ds)

# Export the model to a SavedModel.
model.save("/home/jovyan/notebooks/modelv3")

### Test Spark Pandas Model

In [ ]:
testdf = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/takeoff/BrainFlow-RAW_Recordings_takeoff_4.csv", inferSchema='true')

In [ ]:
tdf = testdf.toPandas()

In [ ]:
tdf.head(2)

In [ ]:
!pip show tensorflow_decision_forests

In [ ]:
import pandas as pd
import tensorflow as tf 
import tensorflow_decision_forests as tfdf

In [ ]:
new_model = tf.keras.models.load_model("/home/jovyan/notebooks/modelv3")

In [ ]:
idk = tfdf.keras.pd_dataframe_to_tf_dataset(tdf)

In [ ]:
idk

In [ ]:
labels = ['backward', 'down', 'forward',
          'land', 'left', 'right', 'takeoff', 'up']

In [ ]:
prediction = np.argmax(prediction, 1)

In [ ]:
predicted_label = labels[prediction[0]]
predicted_label